In [ ]:
# 安裝必要套件
!pip install llama-cpp-python chromadb faiss-cpu PyMuPDF --quiet


In [12]:
# 下載 TinyLLaMA 1.1B Q4_K_M 量化模型 (~700MB)
#!wget -q https://huggingface.co/TheBloke/TinyLlama-1.1B-Chat-v1.0-GGUF/resolve/main/tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf -O tinyllama.gguf
# 下載 Qwen1.5-0.5B-Chat Q4_K_M 量化模型（中文支援）
#!wget -q https://huggingface.co/Qwen/Qwen1.5-0.5B-Chat-GGUF/resolve/main/qwen1_5-0_5b-chat-q4_k_m.gguf -O qwen-0.5b-chat.gguf


In [ ]:
from llama_cpp import Llama

# 載入模型
llm = Llama(model_path="tinyllama.gguf", n_ctx=2048, n_threads=4)
#llm = Llama(model_path="qwen-0.5b-chat.gguf", n_ctx=2048, n_threads=4)
print("模型已載入完成")

In [ ]:
from chromadb import Client
import chromadb.utils.embedding_functions as ef

embedding_func = ef.DefaultEmbeddingFunction()
chroma_client = Client()
collection = chroma_client.create_collection(name="docs", embedding_function=embedding_func)
print("已建立 ChromaDB 檢索資料庫")


In [ ]:
from google.colab import files
import fitz  # PyMuPDF

uploaded = files.upload()
for filename in uploaded.keys():
    if filename.lower().endswith(".pdf"):
        doc = fitz.open(filename)
        text = ""
        for page in doc:
            text += page.get_text()
    elif filename.lower().endswith(".txt"):
        with open(filename, "r", encoding="utf-8") as f:
            text = f.read()
    else:
        print(f"不支援的檔案格式: {filename}")
        continue

    # 分段加入向量資料庫
    chunks = [text[i:i+500] for i in range(0, len(text), 500)]
    ids = [f"{filename}_{i}" for i in range(len(chunks))]
    collection.add(documents=chunks, ids=ids)
    print(f"已處理並加入知識庫: {filename}")

print("文件已全部加入知識庫")


In [22]:
def rag_query(query):
    results = collection.query(query_texts=[query], n_results=3)
    context = " ".join(results["documents"][0])
    prompt = f"""使用以下資料回答問題：
{context}

問題：{query}
回答：
"""
    output = llm(prompt, max_tokens=256, stop=["</s>"])
    return output["choices"][0]["text"]

# 測試
print(rag_query("國家科學及技術委員會中的經費規定"))


Llama.generate: 22 prefix-match hit, remaining 1474 prompt tokens to eval
llama_perf_context_print:        load time =   10062.18 ms
llama_perf_context_print: prompt eval time =   51260.33 ms /  1474 tokens (   34.78 ms per token,    28.76 tokens per second)
llama_perf_context_print:        eval time =   21289.05 ms /   255 runs   (   83.49 ms per token,    11.98 tokens per second)
llama_perf_context_print:       total time =   72706.28 ms /  1729 tokens


一、 備註：
本表僅依訂務單位所屬委員會，均無任何沿遷或項目內容僅限內部參與。
本表是由於該委員會主要工作及經費管理，具有大部分內部交付，其
細節規範，本表項目內容僅依訂務單位所屬委員會項目略稱，其參與
其他委員會條約，僅在必要時，納約沿遷。
二、 考核方式：
本表中各列該費
